In [1]:
%pip install -q openmeteo-requests
%pip install -q requests-cache retry-requests siphon

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openmeteo_requests
import requests_cache
from retry_requests import retry
import subprocess
from google.cloud import bigquery
import sys
from siphon.simplewebservice.ndbc import NDBC
import requests, pandas as pd
from datetime import datetime, timedelta, timezone, time
import os,json
from pathlib import Path
import pytz, folium, gspread
from datetime import datetime
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
from requests.exceptions import HTTPError  # Importer HTTPError
import sys, psycopg2
from urllib.error import HTTPError
from sklearn.preprocessing import StandardScaler
from siphon.simplewebservice.ndbc import NDBC
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, TIMESTAMP, text
from sqlalchemy.exc import ProgrammingError
from urllib.parse import quote_plus

"""
def install_and_import(*packages):
    for package in packages:
        try:
            # Tenter d'importer chaque bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' est déjà installé et importé.")
        except ImportError:
            # Si l'import échoue, installer la bibliothèque
            print(f"Le package '{package}' n'est pas installé. Installation en cours...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            # Ensuite, essayer à nouveau d'importer la bibliothèque
            globals()[package] = __import__(package)
            print(f"Le package '{package}' a été installé et importé.")
 
            
# Utilisation de la fonction pour installer et importer plusieurs packages
install_and_import("dask", "pyowm", "meteomatics","ndbc_api", "plotly", "geopy")

"""
import re

Fonctions

In [ ]:
def drop_columns_if_exist(df, columns_to_drop):
    existing_columns = []
    for col in columns_to_drop:
        if col in df.columns:
            existing_columns.append(col)
            print(f"Colonne '{col}' Supprimée")
        else: 
            print(f"Colonne '{col}' Non Trouvée")
    return df.drop(columns=existing_columns)

def create_schema_and_table(conn, db, schema, table_name):
    # Vérifier si le schéma existe, sinon le créer
    result = conn.execute(text(f"SELECT schema_name FROM information_schema.schemata WHERE schema_name = '{schema}'"))
    if not result.fetchone():
        conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS \"{schema}\""))
        print(f'Schema "{schema}" created.')
    else:
        print(f'Schema "{schema}" already exists.')

    # Vérifier si la table existe, sinon la créer
    result = conn.execute(text(f"SELECT EXISTS (SELECT 1 FROM pg_catalog.pg_tables WHERE schemaname = '{schema}' AND tablename = '{table_name}')"))
    table_exists = result.fetchone()[0]

    if not table_exists:
        print(f"Table '{table_name}' does not exist. Creating...")
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS \"{schema}\".\"{table_name}\" (
            id SERIAL PRIMARY KEY
        );
        """
        conn.execute(text(create_table_query))  # Exécution directe de la requête
        conn.commit()  # S'assurer que la transaction est validée
        print(f"Table '{table_name}' created in schema '{schema}'.")
    else:
        print(f"Table '{table_name}' already exists.")

def add_daytime_and_month_column(df, time_column='Datetime'):
    """
    Ajoute les colonnes 'DayTime' et 'Month' basées sur l'heure et le mois de la colonne 'Datetime' ou d'une autre colonne de type datetime.
    
    Parameters:
    df (pd.DataFrame): La DataFrame sur laquelle ajouter les colonnes.
    time_column (str): Nom de la colonne ou 'index' pour utiliser l'index datetime. Par défaut, utilise l'index.
    
    Returns:
    pd.DataFrame: DataFrame avec les nouvelles colonnes 'DayTime' et 'Month'.
    """
    # Vérifier si la colonne choisie est valide
    if time_column == 'index':
        time_data = df.index
    elif time_column in df.columns and pd.api.types.is_datetime64_any_dtype(df[time_column]):
        time_data = df[time_column]
    else:
        raise ValueError("La colonne spécifiée n'est pas valide ou n'est pas de type datetime.")
    
    # Fonction pour déterminer la période de la journée
    def get_daytime(hour):
        if 6 <= hour < 12:
            return "Morning"
        elif 12 <= hour < 18:
            return "Afternoon"
        elif 18 <= hour < 21:
            return "Evening"
        else:
            return "Night"
    
    # Si on utilise l'index (DatetimeIndex)
    if isinstance(time_data, pd.DatetimeIndex):
        # Extraire l'heure et le mois de l'index datetime
        df['DayTime'] = time_data.hour.map(lambda hour: get_daytime(hour))
        df['Month'] = time_data.month
    elif isinstance(time_data, pd.Series) and pd.api.types.is_datetime64_any_dtype(time_data):
        # Si c'est une colonne datetime classique
        df['DayTime'] = time_data.dt.hour.map(lambda hour: get_daytime(hour))
        df['Month'] = time_data.dt.month
    else:
        raise ValueError("Les données temporelles ne sont pas de type datetime.")

    return df
    
    # Fonction pour déterminer la période de la journée
    def get_daytime(hour):
        if 6 <= hour < 12:
            return "Morning"
        elif 12 <= hour < 18:
            return "Afternoon"
        elif 18 <= hour < 21:
            return "Evening"
        else:
            return "Night"
    # Si on utilise l'index (DatetimeIndex)
    if isinstance(time_data, pd.DatetimeIndex):
        # Extraire l'heure et le mois de l'index datetime
        df['DayTime'] = time_data.hour.map(lambda hour: get_daytime(hour))
        df['Month'] = time_data.month
    else:
        # Si on utilise une colonne datetime classique
        df['DayTime'] = time_data.dt.hour.map(lambda hour: get_daytime(hour))
        df['Month'] = time_data.dt.month
    
    return df

def process_and_resample(df, column_name, resample_interval='h'):
    try:
        # Vérification si la colonne existe dans le DataFrame
        if column_name not in df.columns:
            print(f"Erreur : La colonne '{column_name}' n'existe pas dans le DataFrame.")
            return df

        # Renommer la colonne spécifiée en 'Datetime' et la convertir en datetime
        df.rename(columns={column_name: 'Datetime'}, inplace=True)
        df['Datetime'] = pd.to_datetime(df['Datetime'], errors='raise')

        # Valider l'intervalle de resampling
        valid_intervals = ['h', 'd', 'm', 'w', 'M', 'Y']
        if resample_interval not in valid_intervals:
            print(f"Erreur : L'intervalle de resampling '{resample_interval}' n'est pas valide. Utilisez l'un des suivants : {', '.join(valid_intervals)}.")
            return df

        # Resampler selon l'intervalle choisi et prendre la moyenne
        df_resampled = df.resample(resample_interval).mean().round(2)

        # Obtenir les dates min et max directement depuis l'index
        min_date, max_date = df_resampled.index.min(), df_resampled.index.max()

        # Afficher les dates min et max
        print(f"Plage de dates après resampling ({resample_interval}): \nMin: {min_date}\nMax: {max_date}  ")
        
        # Retourner la DataFrame résultante
        return df_resampled
    
    except ValueError as e:
        print(f"Erreur de conversion des dates : {e}")
        return df
    except KeyError as e:
        print(f"Erreur : La colonne spécifiée n'a pas été trouvée dans le DataFrame. ({e})")
        return df
    except Exception as e:
        print(f"Erreur inattendue : {e}")
        return df
#
def handle_null_values(df):
    # Calcul des pourcentages de valeurs manquantes
    missing_percent = round((df.isnull().sum() / len(df)) * 100, 2)
    missing_percent_str = missing_percent.astype(str) + '%'
    

    # Gestion des valeurs manquantes
    for column in df.columns:
        null_percentage = missing_percent[column]
        
        if null_percentage == 100:
            # Si 100% des valeurs sont manquantes, on supprime la colonne
            print(f"Supprime la colonne : {column} (100% de valeurs manquantes)")
            df = df.drop(column, axis=1)
        elif null_percentage > 50:
            # Si plus de 50% des valeurs sont manquantes, on supprime la colonne
            print(f"Supprime la colonne : {column} ({null_percentage}% de valeurs manquantes)")
            df = df.drop(column, axis=1)
        elif null_percentage > 0:
            # Si moins de 50% des valeurs sont manquantes, on impute avec la médiane
            print(f"Impute la colonne : {column} avec la médiane ({null_percentage}% de valeurs manquantes)")
            median_value = df[column].median()
            df[column] = df[column].fillna(median_value)
       
    # Retourner le DataFrame modifié
    return df
#
def meteo_api_request(coordinates, mode='historical', days=92, interval='hourly'):
    """
    Fonction pour récupérer les données météo depuis l'API Open-Meteo avec cache et réessayer en cas d'erreur.
    
    Paramètres:
        coordinates (list) : liste de coordonnées [latitude, longitude]
        mode (str) : intervalle de données ('forecast' ou 'historical', par défaut 'historical')
        days (int) : nombre de jours dans le passé ou dans le futur (par défaut : 92 pour historique)
        interval (str) : intervalle des données ('hourly' ou 'daily', par défaut 'hourly')

    Retourne :
        pd.DataFrame : un DataFrame avec les données météo
    """
    # Fonction utilitaire pour convertir les coordonnées avec ou sans suffixe (ex: '45.5W', '-45.5')
    def parse_coordinates(coord):
        # Vérifie si la coordonnée a un suffixe de direction (W, E, N, S)
        pattern = r"^([-+]?\d+(\.\d+)?)([NSEW]?)$"
        match = re.match(pattern, str(coord))
        if match:
            value = float(match.group(1))
            direction = match.group(3)
            
            if direction == 'W' or direction == 'S':
                value = -abs(value)  # Si direction est Ouest ou Sud, on inverse la valeur
            
            return value
        else:
            raise ValueError(f"Coordonnée invalide : {coord}")

    # Convertir les coordonnées
    latitude = parse_coordinates(coordinates[0])
    longitude = parse_coordinates(coordinates[1])

    # Setup de l'API client avec retry et cache
    cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
    openmeteo = openmeteo_requests.Client(session=cache_session)

    url = "https://api.open-meteo.com/v1/forecast"
    
    # Paramètres de base
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "past_days": days if mode == 'historical' else None,  # Si historique, utiliser 'past_days'
        "forecast_days": days if mode == 'forecast' else None,  # Si forecast, utiliser 'forecast_days'
        "hourly" if interval.lower() == 'hourly' else "daily": [
            "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
            "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
            "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
            "is_day"
        ]
    }

    # Faire l'appel API
    responses = openmeteo.weather_api(url, params=params)

    # Traiter la réponse pour le premier emplacement
    response = responses[0]  # On prend la première réponse si plusieurs lieux sont fournis

    # Initialisation du dictionnaire pour les données à retourner
    data = {}

    # Processus des données en fonction du mode sélectionné
    if mode == 'historical':
        # Traitement pour données historiques
        if interval == 'hourly':
            hourly = response.Hourly()
            hourly_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="left"
                )
            }
            hourly_variables = [
                "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
                "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
                "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
                "is_day"
            ]
            
            for i, var in enumerate(hourly_variables):
                hourly_data[var] = [round(value, 2) for value in hourly.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(hourly_data)

        elif interval == 'daily':
            daily = response.Daily()
            daily_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="left"
                )
            }
            daily_variables = [
                "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", 
                "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", 
                "precipitation_sum", "rain_sum", "showers_sum", "precipitation_hours", "precipitation_probability_max", 
                "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"
            ]
            
            for i, var in enumerate(daily_variables):
                daily_data[var] = [round(value, 2) for value in daily.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(daily_data)

    # If Forecast is chosen
    elif mode == 'forecast':
        # Traitement pour prévisions
        if interval == 'hourly':
            hourly = response.Hourly()
            hourly_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=hourly.Interval()),
                    inclusive="left"
                )
            }
            hourly_variables = [
                "temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "showers", 
                "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", 
                "cloud_cover_high", "visibility", "wind_speed_10m", "soil_temperature_0cm", "soil_moisture_0_to_1cm", 
                "is_day"
            ]
            
            for i, var in enumerate(hourly_variables):
                hourly_data[var] = [round(value, 2) for value in hourly.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(hourly_data)

        elif interval == 'daily':
            daily = response.Daily()
            daily_data = {
                "date": pd.date_range(
                    start=pd.to_datetime(daily.Time(), unit="s", utc=True),
                    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
                    freq=pd.Timedelta(seconds=daily.Interval()),
                    inclusive="left"
                )
            }
            daily_variables = [
                "temperature_2m_max", "temperature_2m_min", "apparent_temperature_max", "apparent_temperature_min", 
                "sunrise", "sunset", "daylight_duration", "sunshine_duration", "uv_index_max", "uv_index_clear_sky_max", 
                "precipitation_sum", "rain_sum", "showers_sum", "precipitation_hours", "precipitation_probability_max", 
                "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"
            ]
            
            for i, var in enumerate(daily_variables):
                daily_data[var] = [round(value, 2) for value in daily.Variables(i).ValuesAsNumpy()]

            return pd.DataFrame(daily_data)

# Fonction pour créer une base de données si elle n'existe pas
def create_database(dbname, user, password, host, port):
    """Crée une base de données si elle n'existe pas déjà."""
    # Se connecter à la base 'postgres' pour créer d'autres DB
    conn = connect_psycopg2(user=user, password=password, host =host, port=port)
    if conn is None:
        return  # Si la connexion échoue, on arrête la fonction

    conn.autocommit = True  # Désactiver la transaction active
    cur = conn.cursor()
    
    # Vérifier si la base de données existe déjà
    cur.execute(f"SELECT 1 FROM pg_database WHERE datname = '{dbname}'")
    exists = cur.fetchone()

    if not exists:
        try:
            cur.execute(f"CREATE DATABASE {dbname}")
            print(f"✅ Base '{dbname}' créée.")
        except Exception as e:
            print(f"Erreur lors de la création de la base de données : {e}")
    else:
        print(f"ℹ️ La base '{dbname}' existe déjà.")

    cur.close()
    conn.close()

# Fonction pour créer un schéma dans la base
def create_schema(conn, schema_name):
    """Crée un schéma dans la base si ce n'est pas déjà fait."""
    if conn is not None:
        try:
            conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema_name}"))
            print(f"✅ Schéma '{schema_name}' créé.")
        except Exception as e:
            print(f"Erreur lors de la création du schéma : {e}")

In [4]:
# Définir les scopes requis pour accéder à Google Sheets et Google Drive
scopes = [
    'https://spreadsheets.google.com/feeds',
    'https://www.googleapis.com/auth/drive','https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.appdata','https://www.googleapis.com/auth/drive.appfolder'
]

# Chemin vers le fichier de crédentials JSON
path_to_google_cred = r"C:\Users\f.gionnane\Documents\Data Engineering\Credentials\google_credentials.json"

# Créer les crédentials à partir du fichier de service account
creds = Credentials.from_service_account_file(path_to_google_cred, scopes=scopes)

# Autoriser l'accès en utilisant les crédentials
client = gspread.authorize(creds)

# URL de la feuille Google Sheets
sheet_url = "https://docs.google.com/spreadsheets/d/1rXyD4WW5DcrO2iCywfwLbqps3Z-RumxQH9hJofhVM24/edit?gid=0"

# Ouvrir la feuille Google Sheets en utilisant son URL
spreadsheet = client.open_by_url(sheet_url)

worksheets = spreadsheet.worksheets()

drive_service = build('drive', 'v3', credentials=creds)

marine_data_sheet = spreadsheet.get_worksheet(0)  # Accéder à la première feuille (gid=0)
marine_data = marine_data_sheet.get_all_records()  # Récupérer toutes les données de la feuille

meteo_data_sheet = spreadsheet.get_worksheet(0)  # Accéder à la première feuille (gid=0)
meteo_data = marine_data_sheet.get_all_records()  # Récupérer toutes les données de la feuille

print(marine_data)  # Afficher les données

[{'Wind Direction (°)': 50, 'Wind Gusts (km/h)': 7, 'Wave Height (m)': 2.3, 'Average Wave Period (s)': 5.6, 'Dominant Wave Direction (°)': 82, 'Pressure (hPA)': 1011.2, 'Air T°': 26.6, 'Water T°': 27.3, 'dewpoint': 23.5, 'Datetime': '2025-03-06 10:00:00', 'T°(C°)': 26.5, 'Relative Humidity (%)': 81, 'Dew Point (°C)': 22.97, 'Precipitation (mm)': 0, ' Sea Level Pressure (hPa)': 1011.6, 'surface_pressure': 1011.6, 'Low Clouds (%)': 62, 'Middle Clouds (%)': 0, 'High Clouds (%)': 0, 'Visibility (km)': 24.13999939, 'Wind Speed (km/h)': 19.08, 'DayTime': 'Morning', 'Month': 3}, {'Wind Direction (°)': 70, 'Wind Gusts (km/h)': 7, 'Wave Height (m)': 2.3, 'Average Wave Period (s)': 5.6, 'Dominant Wave Direction (°)': 82, 'Pressure (hPA)': 1010.8, 'Air T°': 26.6, 'Water T°': 27.3, 'dewpoint': 23.1, 'Datetime': '2025-03-06 09:00:00', 'T°(C°)': 26.55, 'Relative Humidity (%)': 81, 'Dew Point (°C)': 23.02, 'Precipitation (mm)': 0, ' Sea Level Pressure (hPa)': 1011.5, 'surface_pressure': 1011.5, 'Low 

Get Data from APIs

Marine API

In [5]:
from ndbc_api import NdbcApi
api = NdbcApi()

# get all stations and some metadata as a Pandas DataFrame
stations_df = api.stations()

stations_df.head()

,Station,Hull No./Config and Location,Location Lat/Long,Wind Speed,Wind Direction,Sea Level Pressure,Wave Height,Dominant Period,Air Temp,Water Temp,Dew Point,Remark
0,41001,East Hatteras,34.70N 72.23W,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,"Buoy recovered 10/26/24, data release stopped."
1,41002,3DV33 (SC) South Hatteras,31.75N 74.93W,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Sensor/system failure.,Buoy adrift 12/30/24.
2,41004,3DV02 (SC) Edisto,32.50N 79.08W,100,100,100,100,100,100,99,100,Dewpoint is intermittent.
3,41008,3D36 (SC) Grays Reef,31.40N 80.85W,9,9,100,99,99,9,99,9,All data is intermittent after 8/18/23.
4,41009,3D65 (SC) Canaveral,28.50N 80.18W,100,100,100,31,31,73,38,73,Wave data ceased 3/6/25. Water temperature cea...


Get Nearest

In [6]:
lat_SM = "11.24N"
lon_SM = "74.21W"

nearest = api.nearest_station(lat_SM,lon_SM)
nearest

'42058'

In [7]:
buoy_near_SM = api.station(station_id=nearest, as_df=False)
buoy_near_SM

{'Watch circle radius': '4123 yards',
 'Water depth': '4136 m',
 'Sea temp depth': '1.5 m below water line',
 'Barometer elevation': '2.7 m above mean sea level',
 'Anemometer height': '4.1 m above site elevation',
 'Air temp height': '3.7 m above site elevation',
 'Site elevation': 'sea level',
 'Location': '14.512 N 75.153 W (14°30\'43" N 75°9\'11" W)',
 'Statation Type': 'Owned and maintained by National Data Buoy Center, 3-meter foam buoy, SCOOP payload',
 'Name': 'Station 42058  - Central Caribbean - 210 NM SSE of Kingston, Jamaica'}

Get Data From Json

In [8]:
Buoy_Location = buoy_near_SM['Location']
lat_buoy = float(Buoy_Location.split(' ')[0])  # Convertir en float
lon_buoy = float(Buoy_Location.split(' ')[2])  # Convertir en float

lat_buoy = round(lat_buoy, 2)  # Arrondir à 2 décimales
lon_buoy = round(lon_buoy, 2)

# Extraction du nom de la station et de la zone
station_name = buoy_near_SM['Name'].split('-')[0].strip()  # Utilisez buoy_near_SM['Name']
station_zone = buoy_near_SM['Name'].split('-')[1].strip()

# Création du nom de la table
table_name = f"{station_name.replace(' ', '_')}_{station_zone.replace(' ', '_')}_{lat_buoy}_{lon_buoy}"
table_name = table_name.replace('.', '-')
# Affichage des résultats
print(f'{lat_buoy}, {lon_buoy}, {station_name}, \ntable name : {table_name}')

14.51, 75.15, Station 42058, 
table name : Station_42058_Central_Caribbean_14-51_75-15


In [9]:
from geopy.distance import geodesic
lat_SM = 11.24
lon_SM = -74.21


coord_buoy_1 = (lat_SM, lon_SM)
coord_buoy_2 =(lat_buoy, lon_buoy)


distance_km = geodesic(coord_buoy_1,coord_buoy_2).kilometers

m = folium.Map(location=[lat_SM,lon_SM], zoom_start=6)

folium.Marker(location=[lat_buoy, lon_buoy], popup=f"Buoy nearest to Santa marta ({round(distance_km,2)}km)").add_to(m)
folium.Marker(location=[lat_SM,lon_SM], popup="Santa marta").add_to(m)
m

In [10]:
df_marine = NDBC.realtime_observations(nearest)
# Afficher le résultat
print(df_marine.shape)
df_marine.head()

(6505, 15)


,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean,time
0,80.0,7.0,9.0,NaN,NaN,NaN,NaN,1015.1,27.0,27.5,23.3,NaN,NaN,NaN,2025-03-13 16:10:00+00:00
1,80.0,7.0,9.0,NaN,NaN,NaN,NaN,1015.1,26.9,27.5,23.2,NaN,0.3,NaN,2025-03-13 16:00:00+00:00
2,70.0,7.0,9.0,0.9,6.0,4.0,114.0,1015.3,26.9,27.5,23.1,NaN,NaN,NaN,2025-03-13 15:50:00+00:00
3,80.0,7.0,8.0,NaN,NaN,NaN,NaN,1015.4,27.1,27.5,23.1,NaN,NaN,NaN,2025-03-13 15:40:00+00:00
4,80.0,6.0,7.0,NaN,NaN,NaN,NaN,1015.4,27.2,27.5,23.1,NaN,NaN,NaN,2025-03-13 15:30:00+00:00


In [11]:
import pandas as pd

def process_and_resample(df, column_name, resample_interval='h'):
    try:
        # Vérification si la colonne existe dans le DataFrame
        if column_name not in df.columns:
            print(f"Erreur : La colonne '{column_name}' n'existe pas dans le DataFrame.")
            return df

        # Renommer la colonne spécifiée en 'Datetime' et la convertir en datetime
        df.rename(columns={column_name: 'Datetime'}, inplace=True)
        df['Datetime'] = pd.to_datetime(df['Datetime'], errors='raise')

        # Définir la colonne 'Datetime' comme index
        df.set_index('Datetime', inplace=True)

        # Valider l'intervalle de resampling
        valid_intervals = ['h', 'd', 'm', 'w', 'M', 'Y']
        if resample_interval not in valid_intervals:
            print(f"Erreur : L'intervalle de resampling '{resample_interval}' n'est pas valide. Utilisez l'un des suivants : {', '.join(valid_intervals)}.")
            return df

        # Resampler selon l'intervalle choisi et prendre la moyenne
        df_resampled = df.resample(resample_interval).mean().round(2)

        # Obtenir les dates min et max directement depuis l'index
        min_date, max_date = df_resampled.index.min(), df_resampled.index.max()

        # Afficher les dates min et max
        print(f"Plage de dates après resampling ({resample_interval}): \nMin: {min_date}\nMax: {max_date}  ")

        # Réinitialiser l'index et remettre la colonne 'Datetime' comme colonne normale
        df_resampled.reset_index(inplace=True)

        # Retourner la DataFrame résultante
        return df_resampled
    
    except ValueError as e:
        print(f"Erreur de conversion des dates : {e}")
        return df
    except KeyError as e:
        print(f"Erreur : La colonne spécifiée n'a pas été trouvée dans le DataFrame. ({e})")
        return df
    except Exception as e:
        print(f"Erreur inattendue : {e}")
        return df

# Exemple d'utilisation avec l'intervalle par défaut 'h' (heure)
df_resampled = process_and_resample(df_marine, 'time')  # Intervalle par défaut 'h'
df_resampled.head()

Plage de dates après resampling (h): 
Min: 2025-01-27 00:00:00+00:00
Max: 2025-03-13 16:00:00+00:00  


,Datetime,wind_direction,wind_speed,wind_gust,wave_height,dominant_wave_period,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint,visibility,3hr_pressure_tendency,water_level_above_mean
0,2025-01-27 00:00:00+00:00,70.00,10.17,12.83,2.80,8.0,6.25,85.50,1012.73,27.82,27.75,22.95,NaN,0.6,NaN
1,2025-01-27 01:00:00+00:00,70.00,10.00,12.83,2.87,8.0,6.20,81.67,1013.25,27.83,27.70,23.02,NaN,1.1,NaN
2,2025-01-27 02:00:00+00:00,75.00,9.83,12.50,2.90,8.0,6.35,78.00,1013.72,27.70,27.70,23.42,NaN,1.2,NaN
3,2025-01-27 03:00:00+00:00,70.00,10.67,13.17,2.67,8.0,6.20,85.33,1013.57,27.68,27.70,23.75,NaN,1.4,NaN
4,2025-01-27 04:00:00+00:00,76.67,10.50,12.83,2.90,8.0,6.30,82.50,1013.18,27.72,27.70,23.50,NaN,0.4,NaN


In [12]:
"""

#### Prise en charge de plusieurs buoys à la fois

caribbean_df = stations_df[(stations_df['Lat'] >= 9) & 
                           (stations_df['Lat'] <= 25) & 
                           (stations_df['Lon'] >= -85) & 
                           (stations_df['Lon'] <= -60)]
caribbean_df.shape[0]

buoys_ids = []

for index, row in caribbean_df.iterrows():
    buoys_ids.append(row['Station'])  
print(len(buoys_ids))    

# Liste pour stocker les dataframes
all_dataframes = []

# Compteurs pour les bouées réussies et échouées
successful_buoy = 0
failed_buoy = 0

# Parcours de chaque bouée
for buoy_id in buoys_ids:
    try:
        # Récupère les observations en temps réel pour chaque bouée
        df_buoy = NDBC.realtime_observations(buoy_id)

        # Ajoute le dataframe à la liste
        all_dataframes.append(df_buoy)
        successful_buoy += 1

        # Efface et met à jour la ligne des bouées réussies
        print(f"\r✅ Bouées réussies : {successful_buoy}", end='', flush=True)

    except Exception:
        # Incrémente le compteur des échouées pour toutes les erreurs
        failed_buoy += 1

        # Efface et met à jour la ligne des bouées échouées
        print(f"\r❌ Bouées échouées : {failed_buoy}", end='', flush=True)

# Affichage final propre avec un saut de ligne
print("\n")

# Concaténation des dataframes si disponibles
if all_dataframes:
    final_df = pd.concat(all_dataframes, ignore_index=True)
    print(f"✔️ Concaténation terminée. Nombre de lignes : {final_df.shape[0]} \n Nombre de colonnes : {final_df.shape[1]}")
    print(f"✅ Bouées réussies : {successful_buoy}", end='', flush=True)
    print(f"❌ Bouées échouées : {failed_buoy}", end='', flush=True)

else:
    print("⚠️ Aucune donnée récupérée.")
    
"""

'\n\n#### Prise en charge de plusieurs buoys à la fois\n\ncaribbean_df = stations_df[(stations_df[\'Lat\'] >= 9) & \n                           (stations_df[\'Lat\'] <= 25) & \n                           (stations_df[\'Lon\'] >= -85) & \n                           (stations_df[\'Lon\'] <= -60)]\ncaribbean_df.shape[0]\n\nbuoys_ids = []\n\nfor index, row in caribbean_df.iterrows():\n    buoys_ids.append(row[\'Station\'])  \nprint(len(buoys_ids))    \n\n# Liste pour stocker les dataframes\nall_dataframes = []\n\n# Compteurs pour les bouées réussies et échouées\nsuccessful_buoy = 0\nfailed_buoy = 0\n\n# Parcours de chaque bouée\nfor buoy_id in buoys_ids:\n    try:\n        # Récupère les observations en temps réel pour chaque bouée\n        df_buoy = NDBC.realtime_observations(buoy_id)\n\n        # Ajoute le dataframe à la liste\n        all_dataframes.append(df_buoy)\n        successful_buoy += 1\n\n        # Efface et met à jour la ligne des bouées réussies\n        print(f"\r✅ Bouées ré

In [13]:
# Exemple d'appel de la fonction
df_marine = handle_null_values(df_marine)

print(df_marine.shape)
df_marine.head()

Impute la colonne : wind_direction avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wind_speed avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wind_gust avec la médiane (0.09% de valeurs manquantes)
Impute la colonne : wave_height avec la médiane (43.26% de valeurs manquantes)
Supprime la colonne : dominant_wave_period (67.01% de valeurs manquantes)
Impute la colonne : average_wave_period avec la médiane (43.27% de valeurs manquantes)
Impute la colonne : dominant_wave_direction avec la médiane (43.27% de valeurs manquantes)
Impute la colonne : pressure avec la médiane (0.03% de valeurs manquantes)
Impute la colonne : air_temperature avec la médiane (0.57% de valeurs manquantes)
Impute la colonne : water_temperature avec la médiane (0.77% de valeurs manquantes)
Impute la colonne : dewpoint avec la médiane (0.57% de valeurs manquantes)
Supprime la colonne : visibility (100% de valeurs manquantes)
Supprime la colonne : 3hr_pressure_tendency (83.23% de va

,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint
Datetime,,,,,,,,,,
2025-03-13 16:10:00+00:00,80.0,7.0,9.0,1.7,5.1,81.0,1015.1,27.0,27.5,23.3
2025-03-13 16:00:00+00:00,80.0,7.0,9.0,1.7,5.1,81.0,1015.1,26.9,27.5,23.2
2025-03-13 15:50:00+00:00,70.0,7.0,9.0,0.9,4.0,114.0,1015.3,26.9,27.5,23.1
2025-03-13 15:40:00+00:00,80.0,7.0,8.0,1.7,5.1,81.0,1015.4,27.1,27.5,23.1
2025-03-13 15:30:00+00:00,80.0,6.0,7.0,1.7,5.1,81.0,1015.4,27.2,27.5,23.1


open-meteo API

In [14]:
coordinates = [lat_buoy, lon_buoy]
df_meteo = meteo_api_request(coordinates=coordinates)
print(df_meteo.shape)
df_meteo.head()

(2376, 18)


,date,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-12-11 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2024-12-11 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2024-12-11 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,2024-12-11 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,2024-12-11 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [15]:
df_meteo =process_and_resample(df_meteo, 'date')
df_meteo.head()

Plage de dates après resampling (h): 
Min: 2024-12-11 00:00:00+00:00
Max: 2025-03-19 23:00:00+00:00  


,Datetime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-12-11 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2024-12-11 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2024-12-11 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,2024-12-11 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,2024-12-11 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [16]:
# Arrondir toutes les colonnes numériques à 2 décimales
df_meteo.loc[:, df_meteo.select_dtypes(include=['float32', 'float64']).columns] = df_meteo.select_dtypes(include=['float32', 'float64']).applymap(lambda x: round(x, 2))
df_meteo.head()

,Datetime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,rain,showers,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
0,2024-12-11 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2024-12-11 01:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,2024-12-11 02:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,2024-12-11 03:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,2024-12-11 04:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [17]:
df_meteo = handle_null_values(df_meteo)
print(df_marine.columns)
print(df_meteo.columns)
df_marine.head()

Impute la colonne : temperature_2m avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : relative_humidity_2m avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : dew_point_2m avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : precipitation avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : rain avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : showers avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : pressure_msl avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : surface_pressure avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : cloud_cover avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : cloud_cover_low avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : cloud_cover_mid avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : cloud_cover_high avec la médiane (13.89% de valeurs manquantes)
Impute la colonne : visibilit

,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,dewpoint
Datetime,,,,,,,,,,
2025-03-13 16:10:00+00:00,80.0,7.0,9.0,1.7,5.1,81.0,1015.1,27.0,27.5,23.3
2025-03-13 16:00:00+00:00,80.0,7.0,9.0,1.7,5.1,81.0,1015.1,26.9,27.5,23.2
2025-03-13 15:50:00+00:00,70.0,7.0,9.0,0.9,4.0,114.0,1015.3,26.9,27.5,23.1
2025-03-13 15:40:00+00:00,80.0,7.0,8.0,1.7,5.1,81.0,1015.4,27.1,27.5,23.1
2025-03-13 15:30:00+00:00,80.0,6.0,7.0,1.7,5.1,81.0,1015.4,27.2,27.5,23.1


In [18]:
df_meteo = drop_columns_if_exist(df_meteo,['rain', 'showers','soil_moisture_0_to_1cm', 'cloud_cover', 'soil_temperature_0cm',	'soil_moisture_0_to_1cm', 'is_day'])
df_meteo.head()

Colonne 'rain' Supprimée
Colonne 'showers' Supprimée
Colonne 'soil_moisture_0_to_1cm' Supprimée
Colonne 'cloud_cover' Supprimée
Colonne 'soil_temperature_0cm' Supprimée
Colonne 'soil_moisture_0_to_1cm' Supprimée
Colonne 'is_day' Supprimée


,Datetime,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,pressure_msl,surface_pressure,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m
0,2024-12-11 00:00:00+00:00,23.625,56.5,14.25,0.0,1012.7,945.45,0.0,0.0,0.0,24140.0,6.88
1,2024-12-11 01:00:00+00:00,23.625,56.5,14.25,0.0,1012.7,945.45,0.0,0.0,0.0,24140.0,6.88
2,2024-12-11 02:00:00+00:00,23.625,56.5,14.25,0.0,1012.7,945.45,0.0,0.0,0.0,24140.0,6.88
3,2024-12-11 03:00:00+00:00,23.625,56.5,14.25,0.0,1012.7,945.45,0.0,0.0,0.0,24140.0,6.88
4,2024-12-11 04:00:00+00:00,23.625,56.5,14.25,0.0,1012.7,945.45,0.0,0.0,0.0,24140.0,6.88


In [19]:
df_meteo.rename(columns={'temperature_2m': 'T°(C°)', 
                         'relative_humidity_2m': 'Relative Humidity (%)',
                         'dew_point_2m': 'Dew Point (°C)', 
                         'precipitation': 'Precipitation (mm)', 
                         'pressure_msl':' Sea Level Pressure (hPa)', 
                         'cloud_cover_low':'Low Clouds (%)',
                         'cloud_cover_mid' : 'Middle Clouds (%)',	
                         'cloud_cover_high' : 'High Clouds (%)', 
                         'visibility' : ' Visibility (%)', 
                         'wind_speed_10m' : 'Wind Speed (km/h)'}, 
                         inplace=True)
df_marine.rename(columns={
    'wind_direction': 'Wind Direction (°)',
    'wind_speed': 'Wind Speed (km/h)',
    'wind_gust': 'Wind Gusts (km/h)',
    'wave_height': 'Wave Height (m)',
    'average_wave_period': 'Average Wave Period (s)',
    'dominant_wave_direction': 'Dominant Wave Direction (°)',
    'pressure': 'Pressure (hPA)',
    'air_temperature': 'Air T°',
    'water_temperature': 'Water T°'}, 
    inplace=True)

print(df_meteo.columns)
print(df_marine.columns)
print(df_marine.shape)
print(df_meteo.shape)

Index(['Datetime', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitation (mm)', ' Sea Level Pressure (hPa)', 'surface_pressure',
       'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
       ' Visibility (%)', 'Wind Speed (km/h)'],
      dtype='object')
Index(['Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPA)', 'Air T°', 'Water T°',
       'dewpoint'],
      dtype='object')
(6505, 10)
(2376, 12)


Merging Dataframes

In [20]:
# Effectuer la jointure interne sur la colonne 'time'
df_merged = pd.merge(df_marine, df_meteo, on = 'Datetime', how='inner')

# Afficher le résultat
print(df_merged.shape)
print(df_merged.dtypes)
df_merged.head(20)

(1096, 22)
Datetime                       datetime64[ns, UTC]
Wind Direction (°)                         float64
Wind Speed (km/h)_x                        float64
Wind Gusts (km/h)                          float64
Wave Height (m)                            float64
Average Wave Period (s)                    float64
Dominant Wave Direction (°)                float64
Pressure (hPA)                             float64
Air T°                                     float64
Water T°                                   float64
dewpoint                                   float64
T°(C°)                                     float64
Relative Humidity (%)                      float32
Dew Point (°C)                             float64
Precipitation (mm)                         float64
 Sea Level Pressure (hPa)                  float64
surface_pressure                           float64
Low Clouds (%)                             float32
Middle Clouds (%)                          float32
High Clouds (%)     

,Datetime,Wind Direction (°),Wind Speed (km/h)_x,Wind Gusts (km/h),Wave Height (m),Average Wave Period (s),Dominant Wave Direction (°),Pressure (hPA),Air T°,Water T°,...,Relative Humidity (%),Dew Point (°C),Precipitation (mm),Sea Level Pressure (hPa),surface_pressure,Low Clouds (%),Middle Clouds (%),High Clouds (%),Visibility (%),Wind Speed (km/h)_y
0,2025-03-13 16:00:00+00:00,80.0,7.0,9.0,1.7,5.1,81.0,1015.1,26.9,27.5,...,49.0,16.02,0.0,1010.5,944.36,0.0,0.0,0.0,24140.0,2.90
1,2025-03-13 15:00:00+00:00,80.0,7.0,8.0,1.7,5.1,81.0,1015.4,27.2,27.5,...,49.0,17.03,0.0,1009.9,944.03,0.0,2.0,0.0,24140.0,1.61
2,2025-03-13 14:00:00+00:00,80.0,8.0,9.0,1.7,5.1,81.0,1015.1,27.1,27.4,...,44.0,15.88,0.0,1008.8,943.13,25.0,0.0,0.0,24140.0,3.62
3,2025-03-13 13:00:00+00:00,80.0,8.0,9.0,1.7,5.1,81.0,1014.8,27.1,27.4,...,33.0,13.29,0.0,1008.1,942.91,6.0,0.0,0.0,24140.0,8.31
4,2025-03-13 12:00:00+00:00,80.0,8.0,9.0,1.7,5.1,81.0,1014.2,27.0,27.4,...,21.0,8.48,0.0,1006.9,942.26,0.0,0.0,0.0,24140.0,12.73
5,2025-03-13 11:00:00+00:00,70.0,7.0,8.0,1.7,5.1,81.0,1013.3,26.9,27.4,...,18.0,6.71,0.0,1006.6,942.10,0.0,0.0,0.0,24140.0,14.62
6,2025-03-13 10:00:00+00:00,70.0,7.0,9.0,1.7,5.1,81.0,1012.6,26.9,27.4,...,27.0,12.84,0.0,1007.4,942.87,3.0,0.0,5.0,24140.0,14.33
7,2025-03-13 09:00:00+00:00,60.0,9.0,11.0,1.7,5.1,81.0,1012.1,26.9,27.5,...,34.0,16.06,0.0,1008.5,943.82,29.0,0.0,97.0,24140.0,11.72
8,2025-03-13 08:00:00+00:00,70.0,7.0,9.0,1.7,5.1,81.0,1011.9,26.8,27.5,...,40.0,17.61,0.0,1010.2,945.18,0.0,0.0,100.0,24140.0,6.48
9,2025-03-13 07:00:00+00:00,60.0,8.0,9.0,1.7,5.1,81.0,1012.0,26.9,27.5,...,46.0,18.67,0.0,1011.6,946.22,0.0,0.0,100.0,24140.0,4.32


In [21]:
# Exemple d'utilisation
df_merged = add_daytime_and_month_column(df_merged,'Datetime')

print(df_merged.columns)
df_merged.head(10)

Index(['Datetime', 'Wind Direction (°)', 'Wind Speed (km/h)_x',
       'Wind Gusts (km/h)', 'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPA)', 'Air T°', 'Water T°',
       'dewpoint', 'T°(C°)', 'Relative Humidity (%)', 'Dew Point (°C)',
       'Precipitation (mm)', ' Sea Level Pressure (hPa)', 'surface_pressure',
       'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
       ' Visibility (%)', 'Wind Speed (km/h)_y', 'DayTime', 'Month'],
      dtype='object')


,Datetime,Wind Direction (°),Wind Speed (km/h)_x,Wind Gusts (km/h),Wave Height (m),Average Wave Period (s),Dominant Wave Direction (°),Pressure (hPA),Air T°,Water T°,...,Precipitation (mm),Sea Level Pressure (hPa),surface_pressure,Low Clouds (%),Middle Clouds (%),High Clouds (%),Visibility (%),Wind Speed (km/h)_y,DayTime,Month
0,2025-03-13 16:00:00+00:00,80.0,7.0,9.0,1.7,5.1,81.0,1015.1,26.9,27.5,...,0.0,1010.5,944.36,0.0,0.0,0.0,24140.0,2.90,Afternoon,3
1,2025-03-13 15:00:00+00:00,80.0,7.0,8.0,1.7,5.1,81.0,1015.4,27.2,27.5,...,0.0,1009.9,944.03,0.0,2.0,0.0,24140.0,1.61,Afternoon,3
2,2025-03-13 14:00:00+00:00,80.0,8.0,9.0,1.7,5.1,81.0,1015.1,27.1,27.4,...,0.0,1008.8,943.13,25.0,0.0,0.0,24140.0,3.62,Afternoon,3
3,2025-03-13 13:00:00+00:00,80.0,8.0,9.0,1.7,5.1,81.0,1014.8,27.1,27.4,...,0.0,1008.1,942.91,6.0,0.0,0.0,24140.0,8.31,Afternoon,3
4,2025-03-13 12:00:00+00:00,80.0,8.0,9.0,1.7,5.1,81.0,1014.2,27.0,27.4,...,0.0,1006.9,942.26,0.0,0.0,0.0,24140.0,12.73,Afternoon,3
5,2025-03-13 11:00:00+00:00,70.0,7.0,8.0,1.7,5.1,81.0,1013.3,26.9,27.4,...,0.0,1006.6,942.10,0.0,0.0,0.0,24140.0,14.62,Morning,3
6,2025-03-13 10:00:00+00:00,70.0,7.0,9.0,1.7,5.1,81.0,1012.6,26.9,27.4,...,0.0,1007.4,942.87,3.0,0.0,5.0,24140.0,14.33,Morning,3
7,2025-03-13 09:00:00+00:00,60.0,9.0,11.0,1.7,5.1,81.0,1012.1,26.9,27.5,...,0.0,1008.5,943.82,29.0,0.0,97.0,24140.0,11.72,Morning,3
8,2025-03-13 08:00:00+00:00,70.0,7.0,9.0,1.7,5.1,81.0,1011.9,26.8,27.5,...,0.0,1010.2,945.18,0.0,0.0,100.0,24140.0,6.48,Morning,3
9,2025-03-13 07:00:00+00:00,60.0,8.0,9.0,1.7,5.1,81.0,1012.0,26.9,27.5,...,0.0,1011.6,946.22,0.0,0.0,100.0,24140.0,4.32,Morning,3


In [22]:
df_merged['Wind Speed (km/h)'] = (df_merged['Wind Speed (km/h)_x']+ df_merged['Wind Speed (km/h)_y'])/2
df_merged = drop_columns_if_exist(df_merged, ['Wind Speed (km/h)_x', 'Wind Speed (km/h)_y', 'Wind Gusts (km/h)'])

print(df_merged.columns)
df_merged.head()

Colonne 'Wind Speed (km/h)_x' Supprimée
Colonne 'Wind Speed (km/h)_y' Supprimée
Colonne 'Wind Gusts (km/h)' Supprimée
Index(['Datetime', 'Wind Direction (°)', 'Wave Height (m)',
       'Average Wave Period (s)', 'Dominant Wave Direction (°)',
       'Pressure (hPA)', 'Air T°', 'Water T°', 'dewpoint', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitation (mm)',
       ' Sea Level Pressure (hPa)', 'surface_pressure', 'Low Clouds (%)',
       'Middle Clouds (%)', 'High Clouds (%)', ' Visibility (%)', 'DayTime',
       'Month', 'Wind Speed (km/h)'],
      dtype='object')


,Datetime,Wind Direction (°),Wave Height (m),Average Wave Period (s),Dominant Wave Direction (°),Pressure (hPA),Air T°,Water T°,dewpoint,T°(C°),...,Precipitation (mm),Sea Level Pressure (hPa),surface_pressure,Low Clouds (%),Middle Clouds (%),High Clouds (%),Visibility (%),DayTime,Month,Wind Speed (km/h)
0,2025-03-13 16:00:00+00:00,80.0,1.7,5.1,81.0,1015.1,26.9,27.5,23.2,27.70,...,0.0,1010.5,944.36,0.0,0.0,0.0,24140.0,Afternoon,3,4.950
1,2025-03-13 15:00:00+00:00,80.0,1.7,5.1,81.0,1015.4,27.2,27.5,22.9,28.80,...,0.0,1009.9,944.03,0.0,2.0,0.0,24140.0,Afternoon,3,4.305
2,2025-03-13 14:00:00+00:00,80.0,1.7,5.1,81.0,1015.1,27.1,27.4,22.7,29.40,...,0.0,1008.8,943.13,25.0,0.0,0.0,24140.0,Afternoon,3,5.810
3,2025-03-13 13:00:00+00:00,80.0,1.7,5.1,81.0,1014.8,27.1,27.4,22.6,31.50,...,0.0,1008.1,942.91,6.0,0.0,0.0,24140.0,Afternoon,3,8.155
4,2025-03-13 12:00:00+00:00,80.0,1.7,5.1,81.0,1014.2,27.0,27.4,22.5,33.85,...,0.0,1006.9,942.26,0.0,0.0,0.0,24140.0,Afternoon,3,10.365


Connexion BigQuery

In [26]:
# from google.oauth2 import service_account
# from google.cloud import storage  # Exemple pour Google Cloud Storage
# from google.cloud import bigquery
# from google.cloud.exceptions import NotFound
# import pandas as pd
# import pyarrow

# path_to_google_creds = r"C:\Users\f.gionnane\Documents\Data Engineering\Credentials\google_credentials.json"

# bq_client = bigquery.Client.from_service_account_json(
#     path_to_google_creds)
# project_id = "rare-bloom-419220"
# dataset_End_To_End_Oceanography_ML = "End_To_End_Oceanography_ML"
# bq_client
# # dataset_ref = bq_client.dataset('my_dataset_name', project=project_id)


# # LIST DATASETS AND FIND ONE
# datasets = list(bq_client.list_datasets())  # Make an API request.
# project = client.project
# bq_datasets_list =[]

# if datasets:
#     print("Datasets in project {}:".format(project))
#     for dataset in datasets:
#         print("\t{}".format(dataset.dataset_id))
#         bq_datasets_list.append(dataset.dataset_id)
#     if dataset_End_To_End_Oceanography_ML in bq_datasets_list:
#         dataset =  dataset_End_To_End_Oceanography_ML
#         print("Dataset Found !")
# else:
#     print("{} project does not contain any datasets.".format(project))

# # (developer): Set table_id to the ID of the table to determine existence.
# # table_id = "your-project.your_dataset.your_table"

# try:
#     table_ref = bq_client.dataset(dataset).table(table_name)
#     bq_client.get_table(table_ref)  # Make an API request.
#     print("Table {} already exists.".format(table_name))
# except NotFound:
#     print("Table {} is not found.".format(table_name))


# def clean_column_names(df):

#     cleaned_columns = []
#     for column in df.columns:
#         # Remplacer tous les caractères non alphanumériques (sauf underscores) par un underscore
#         cleaned_column = re.sub(r'[^A-Za-z0-9_]', '_', column)
        
#         # Ajouter le nom de colonne nettoyé à la liste
#         cleaned_columns.append(cleaned_column)
    
#     # Appliquer les nouveaux noms de colonnes au DataFrame
#     df.columns = cleaned_columns
#     return df

# df_merged = clean_column_names(df_merged)

# table_id = f"{project_id}.{dataset}.{table_name}"


# try:
#     bq_client.get_table(table_id)  # Make an API request.
#     print("Table {} already exists.".format(table_id))
# except NotFound:
#     print("Table {} is not found.".format(table_id))
#     bq_client.create_table(table_id)
#     print("Creation of the Table {}.".format(table_id))


# def load_data_to_bigquery(client, dataset: str = None, table: str = None, df: pd.DataFrame = None, key_column: str = 'Datetime', table_id: str = None):
   
    
#     # Fonction pour détecter et convertir les types de données
#     def convert_column_types(df):
#         for column in df.columns:
#             dtype = df[column].dtype

#             if dtype == 'object':  # Chaînes de caractères
#                 df[column] = df[column].astype(str)
#             elif dtype == 'datetime64[ns]':  # Datetime
#                 df[column] = pd.to_datetime(df[column], errors='coerce').dt.tz_localize('UTC', ambiguous='NaT').dt.tz_localize(None)
#             elif dtype == 'float64':  # Float
#                 df[column] = df[column].astype('float32')
#             elif dtype == 'int64':  # Integer
#                 df[column] = df[column].astype('int64')  # On garde int64, car BigQuery supporte ce type
#             else:
#                 # Autres types, on les convertit en string
#                 df[column] = df[column].astype(str)
        
#         return df

#     # Convertir les types des colonnes
#     df = convert_column_types(df)

#     if table_id:
#         # Si table_id est fourni, on l'utilise directement.
#         full_table_id = table_id
#     elif dataset and table:
#         # Si table_id n'est pas fourni, on construit table_id à partir de dataset et table.
#         full_table_id = f"{client.project}.{dataset}.{table}"
#     else:
#         raise ValueError("Il faut fournir soit 'table_id' ou les paramètres 'dataset' et 'table' séparés.")

#     # Vérifier si le dataset existe, sinon le créer
#     try:
#         dataset = full_table_id.split('.')[1]
#         client.get_dataset(dataset)  # Vérifie si le dataset existe
#         print(f"Le dataset {dataset} existe déjà.")
#     except NotFound:
#         print(f"Le dataset {dataset} n'existe pas. Création du dataset...")
#         client.create_dataset(dataset)  # Crée le dataset s'il n'existe pas
#         print(f"Le dataset {dataset} a été créé.")

#     # Vérifier si la table existe, sinon la créer
#     try:
#         client.get_table(full_table_id)  # Vérifie si la table existe
#         print(f"La table {full_table_id} existe déjà.")
#     except NotFound:
#         print(f"La table {full_table_id} n'existe pas. Création de la table...")
#         # Créer la table avec le schéma du DataFrame
#         schema = []
#         for name, dtype in df.dtypes.items():
#             if name == 'Datetime':
#                 schema.append(bigquery.SchemaField(name, bigquery.enums.SqlTypeNames.TIMESTAMP))
#             elif dtype == 'float32' or dtype == 'float64':
#                 schema.append(bigquery.SchemaField(name, bigquery.enums.SqlTypeNames.FLOAT64))
#             elif dtype == 'int64':
#                 schema.append(bigquery.SchemaField(name, bigquery.enums.SqlTypeNames.INTEGER))
#             else:
#                 schema.append(bigquery.SchemaField(name, bigquery.enums.SqlTypeNames.STRING))
        
#         # Créer la table avec le schéma
#         table = bigquery.Table(full_table_id, schema=schema)
#         client.create_table(table)  # Crée la table si elle n'existe pas
#         print(f"La table {full_table_id} a été créée.")

#     # Préparer les données pour l'insertion
#     if key_column in df.columns:
#         # Si la colonne clé est fournie, supprimer les doublons en fonction de cette colonne
#         df = df.drop_duplicates(subset=[key_column])

#     # Charger les données dans la table BigQuery
#     job_config = bigquery.LoadJobConfig(
#         schema=[
#             bigquery.SchemaField(name, bigquery.enums.SqlTypeNames.TIMESTAMP) if name == 'Datetime' else
#             bigquery.SchemaField(name, bigquery.enums.SqlTypeNames.FLOAT64) if dtype == 'float32' or dtype == 'float64' else
#             bigquery.SchemaField(name, bigquery.enums.SqlTypeNames.INTEGER) if dtype == 'int64' else
#             bigquery.SchemaField(name, bigquery.enums.SqlTypeNames.STRING)
#             for name, dtype in df.dtypes.items()
#         ],
#         write_disposition="WRITE_APPEND"  # Ajoute les données sans écraser les anciennes
#     )

#     # Charger le DataFrame dans BigQuery
#     job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
#     job.result()  # Attendre la fin de la tâche
#     print(f"Données chargées dans la table {full_table_id}.")

# # Exemple d'appel à la fonction
# load_data_to_bigquery(table_id=table_id, client=bq_client, df=df_merged)


PostgreSQL

In [ ]:
from sqlalchemy import create_engine, text
import json

# Charger les informations de connexion PostgreSQL
path_postgresql_creds = r"C:\Users\f.gionnane\Documents\Data Engineering\Credentials\postgresql_creds.json"
with open(path_postgresql_creds, 'r') as file:
    content = json.load(file)
    user = content["user"]
    password = content["password"]
    host = content["host"]
    port = content["port"]

db = "MyProjects"
schema = "End_To_End_Oceanography_ML"

# Créer l'engine PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")
conn = engine.connect()

def create_schema_and_table(conn, db, schema, table_name):
    # Vérifier si le schéma existe, sinon le créer
    result = conn.execute(text(f"SELECT schema_name FROM information_schema.schemata WHERE schema_name = '{schema}'"))
    if not result.fetchone():
        conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS \"{schema}\""))
        print(f'Schema "{schema}" created.')
    else:
        print(f'Schema "{schema}" already exists.')

    # Vérifier si la table existe, sinon la créer
    result = conn.execute(text(f"SELECT EXISTS (SELECT 1 FROM pg_catalog.pg_tables WHERE schemaname = '{schema}' AND tablename = '{table_name}')"))
    table_exists = result.fetchone()[0]

    if not table_exists:
        print(f"Table '{table_name}' does not exist. Creating...")
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS \"{schema}\".\"{table_name}\" (
            id SERIAL PRIMARY KEY
        );
        """
        conn.execute(text(create_table_query))  # Exécution directe de la requête
        conn.commit()  # S'assurer que la transaction est validée
        print(f"Table '{table_name}' created in schema '{schema}'.")
    else:
        print(f"Table '{table_name}' already exists.")

# Créer l'engine PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")

# Connexion à la base de données
create_schema_and_table(conn, db, schema, table_name)

Schema "End_To_End_Oceanography_ML" already exists.
Table 'Station_42058_Central_Caribbean_14-51_75-15' already exists.
